# 3d mitochondria visualizer with synapses
This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409611,2864896,3878784,93134,59860,836,93000,59406,798,93294,60294,867,648518346349536888,333419.72,214298.80,33440
2409612,2161774,4075748,80312,56522,1814,79536,55348,1714,80940,57054,1889,648518346349524070,287516.96,202348.76,72560
2409613,2753701,4576444,90324,60436,2037,89490,60120,1851,90750,61138,2142,648518346341354380,323359.92,216360.88,81480
2409614,1963708,5805612,75674,72546,744,75266,71628,670,76032,73302,795,648518346343047176,270912.92,259714.68,29760


# Enter a cellid of interest

In [6]:
# pull all mitos from a cellid of interest
cellid = 648518346341378315

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf

# interesting cellids
# 648518346349530724 microglia with 149 mitos
# 648518346349527319 astrocyte containing the largest mito in the volume and several additional large mitos
# 648518346349537555 apical dendrite likely from a L5 or L6 pyr neuron
# 648518346349537741 pyr neuron with largest mito in the volume for a neuron
# 648518346349508279 oligodendrocyte
# 648518346349539853 one of three neurons in the default view of Neuroglancer
# 648518346349539376 pyr neuron with large apical dendritic arbor
# 648518346349536391 partial pyr neuron with basal dendrite in volume
# 648518346349539934 pyr neuron
# 648518346349525715 astrocyte
# 648518346349516236 pyr spiny dendrite in contact with 648518346349537982
# 648518346349537982 very large spiny dendrite (L5?)
# 648518346349537988 continuation of process as 648518346349537982 but was not continuously segmented
# 648518346349489861 bipolar inhibitor neuron with axon-carrying dendrite
# 648518346341399075 large spiny dendrite with portion near the largest mito in volume which belongs to astro 648518346349527319
# 648518346341368034 bizarre large dendritic process, sparsely spined, perhaps L5 Near Projecting (NP) pyr neuron or a bipolar neuron

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
8998,1184875,875200,64358,36172,1311,63630,35270,1253,64936,36778,1354,648518346341378315,230401.64,129495.76,52440
27549,1077378,16716824,62696,51452,1636,60842,36844,1136,64652,70270,2157,648518346341378315,224451.68,184198.16,65440
29004,961802,372228,60506,36074,1611,60304,35268,1584,60854,36756,1632,648518346341378315,216611.48,129144.92,64440
74010,1115710,11696,62802,54190,1645,62786,54120,1643,62824,54274,1647,648518346341378315,224831.16,194000.20,65800
93943,1110906,99156,62738,52584,1618,62652,52472,1607,62830,52672,1629,648518346341378315,224602.04,188250.72,64720
128262,962219,33612,60778,36722,1601,60742,36656,1596,60812,36778,1608,648518346341378315,217585.24,131464.76,64040
198300,1095532,3672,61194,45252,1272,61168,45228,1269,61232,45274,1275,648518346341378315,219074.52,162002.16,50880
241812,1235424,40456,63760,59366,1844,63632,59176,1833,63858,59594,1853,648518346341378315,228260.80,212530.28,73760
262076,1105790,267460,62526,50928,1589,62238,49918,1563,62874,51944,1606,648518346341378315,223843.08,182322.24,63560
306944,1231001,108088,63474,57018,1754,63428,56830,1742,63514,57230,1770,648518346341378315,227236.92,204124.44,70160


In [7]:
mito_query_mitolist = mito_querydf.mito_id.to_list()
print(f"length: "+str(len(mito_query_mitolist)))
print(f"type: "+str(type(mito_query_mitolist)))
print('')
# print(mito_query_mitolist) # uncomment to print the mito id list

length: 26
type: <class 'list'>



# vtk 3d viewer

In [8]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://td.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [9]:
cell_id = cellid

In [10]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [11]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

In [12]:
len(mito_query_mitolist)

26

In [13]:
mito_query_mitolist[0]

1184875

### The following cell only needs to be run once
- You do not need to run this cell if you have previously downloaded the mito mesh files
- Note: this will take a long time, which gets longer the more mitos are in the cell of interest

In [14]:
# download the mito meshes for this cell

for i in range(len(mito_query_mitolist)):
    mito_id = mito_query_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)

In [15]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator = {}
for i in range(len(mito_query_mitolist)):
    mito_seg_id = mito_query_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator)

In [16]:
cell_opac = 0.075
cell_color = (0.2, 0.4, 0.7)
mito_opac = 0.35

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator = {}
for i in range(len(mito_query_mitolist)):
    random_color = list(np.random.random(size=3))
    actor_iterator['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator['mito_mesh_'+str(i)], opacity=mito_opac, color=(random_color))
    
locals().update(actor_iterator)

# update dictionary for the render actors code below
actor_iterator['mesh_actor'] = mesh_actor

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=150)

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001EA44E080D0) at 0x000001EA47B18708>

In [17]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )

# Add synapses to visualization
- Download synapse table from https://www.microns-explorer.org/phase1

In [18]:
syn = pd.read_csv('data/221206_pni_synapses_v185.csv')
syn

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
0,2477272,648518346349538285,648518346349536759,1103,280784,261896,40600,70050,65534,1015,70196,65474,1015,70104,65602,1015
1,4538436,648518346342404863,648518346342414574,62,365832,292808,25800,91472,73196,645,91458,73202,645,91396,73216,645
2,1124039,648518346342921567,648518346345323296,62,229144,155952,48080,57250,39028,1202,57286,38988,1202,57312,38966,1203
3,5245264,648518346342796822,648518346349537255,955,398452,208560,50760,99538,52184,1276,99613,52140,1269,99550,52098,1276
4,6928613,648518346342800412,648518346348190542,718,461868,245212,53560,115462,61350,1335,115467,61303,1339,115468,61296,1335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239270,2167340,648518346348045782,648518346349537840,106,268776,173356,14360,67190,43302,361,67194,43339,359,67216,43364,360
3239271,1944654,648518346347834064,648518346349537840,50,267344,156520,8520,66762,39152,213,66836,39130,213,66874,39134,213
3239272,2749491,648518346347876810,648518346349537840,139,292772,156680,12600,73136,39170,315,73193,39170,315,73218,39160,315
3239273,6924909,648518346348343883,648518346349537840,987,462768,237776,8520,115680,59370,208,115692,59444,213,115628,59430,214


In [19]:
# filter all the pre- and post- synapses for this cell
cell_post_subgraph = syn.query(f'post_root_id=={seg_id}') #this shows all post-syn sites on this seg_id cell
cell_pre_subgraph = syn.query(f'pre_root_id=={seg_id}') #this one shows all pre-syn sites on the axon of seg_id cell

# make an array of xyz synapse positions
postsyn_xyz=cell_post_subgraph[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values
presyn_xyz=cell_pre_subgraph[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values

In [20]:
len(postsyn_xyz)

296

In [21]:
len(presyn_xyz)

1

In [22]:
# Make a list of the pre_root_ids that form a synapse on this cell
cell_post_subgraph

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
2558,1826949,648518346341378140,648518346341378315,61,252360,235088,73320,63038,58766,1833,63090,58772,1833,63136,58748,1833
14241,1587050,648518346344499162,648518346341378315,192,249360,203168,63080,62270,50780,1579,62340,50792,1577,62342,50828,1579
16539,1868818,648518346341380755,648518346341378315,63,254104,265672,79880,63544,66432,1996,63526,66418,1997,63474,66388,1996
19496,1881185,648518346341375950,648518346341378315,64,257560,273688,84200,64394,68392,2106,64390,68422,2105,64326,68414,2106
21428,1841195,648518346341360930,648518346341378315,1185,256880,238652,72800,64180,59750,1821,64220,59663,1820,64102,59714,1821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2787716,1587529,648518346344512732,648518346341378315,1421,250224,200972,60800,62480,50272,1510,62556,50243,1520,62460,50274,1513
2797804,1730234,648518346341412275,648518346341378315,927,255964,150512,54840,63950,37648,1375,63991,37628,1371,63958,37568,1375
2799645,1558116,648518346342916673,648518346341378315,60,249696,179296,57880,62462,44816,1448,62424,44824,1447,62384,44844,1447
2815794,1817574,648518346349497669,648518346341378315,108,252000,228556,69800,62950,57128,1743,63000,57139,1745,62956,57118,1746


In [23]:
# Uncomment if you want to generate a list for copy and pasting
# cell_post_subgraph["pre_root_id"].to_list()

In [24]:
# Make a list of the post_root_ids that receive a synapse from this cell
cell_pre_subgraph

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
289333,1566372,648518346341378315,648518346349538201,96,247208,184120,53760,61742,45998,1345,61802,46030,1344,61842,45982,1345


In [25]:
# Uncomment if you want to generate a list for copy and pasting
# cell_pre_subgraph["post_root_id"].to_list()

In [26]:
# settings to visualize pre and post synaptic sites (as dots)
pre_color = (0.2, 0.9, 0.2)
post_color = (0.9, 0.2, 0.2)
pre_opac = 0.25
post_opac = 0.25
pre_size = 250
post_size = 200
presyn_actor = trimesh_vtk.point_cloud_actor(presyn_xyz, size=pre_size, opacity=pre_opac, color=pre_color)  
postsyn_actor = trimesh_vtk.point_cloud_actor(postsyn_xyz, size=post_size, opacity=post_opac, color=post_color)

# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.05
cell_color = (0.2, 0.4, 0.7)
mito_opac = 0.35

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator = {}
for i in range(len(mito_query_mitolist)):
    random_color = list(np.random.random(size=3))
    actor_iterator['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator['mito_mesh_'+str(i)], opacity=mito_opac, color=(random_color))
    
locals().update(actor_iterator)

# update dictionary for the render actors code below
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator['presyn_actor'] = presyn_actor
actor_iterator['postsyn_actor'] = postsyn_actor

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=150)

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)



setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001EA55536A90) at 0x000001EA47AB7B28>

In [27]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )

In [28]:
actor_iterator

{'mito_mesh_actor_0': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001EA41D73390) at 0x000001EA0389D468>,
 'mito_mesh_actor_1': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001EA41D735C0) at 0x000001EA0389DA68>,
 'mito_mesh_actor_2': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001EA41D707D0) at 0x000001EA0389DB28>,
 'mito_mesh_actor_3': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001EA41D6C630) at 0x000001EA0389DB88>,
 'mito_mesh_actor_4': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001EA41D6FF10) at 0x000001EA0389DBE8>,
 'mito_mesh_actor_5': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001EA41D70140) at 0x000001EA0389DC48>,
 'mito_mesh_actor_6': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001EA41D74510) at 0x000001EA0389DCA8>,
 'mito_mesh_actor_7': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001EA41D74740) at 0x000001EA0389DD08>,
 'mito_mesh_actor_8': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001EA5C7714A0) at 0x00